In [ ]:
#Imports
import functions as functions
import numpy as np

## Import elements (steps: A., B., C. of the methodology $\mathfrak{M}()$)

In [ ]:
elements = functions.import_elements('../Data/Elements.xlsx')

## Identify hypothetical paths compatible with the well-defined information, $\mathcal{H}^{\mathcal{I}'}$ using the A* algorithm (step: D. of the methodology $\mathfrak{M}()$)

In [ ]:
paths = []
for _, elem in elements[elements["type"]=="customer"].iterrows():
    p = functions.astar(elements, elem.squeeze(), num_paths=3, D=20)
    paths.extend(p)

### Print the elements in each path

In [ ]:
for p in paths:
    print(p.get_names())
print(f"Found {len(paths)} paths")

### Matrix generation

In [ ]:
H = functions.BuildHMatrix(paths, elements)

C = functions.Subset(elements, ["customer"])
T = functions.Subset(elements, ["junction"])
R = functions.Subset(elements, ["line"])

Hc, Hr, Ht = functions.MatrixDivision(elements, H, C,R,T)

## Optimization problem (step: E. of the methodology $\mathfrak{M}()$)

In [ ]:
model, Tr, P = functions.optimization_problem(C,R,T, H,Hc,Hr,Ht)

### Results explanation

#### Matrices $\hat{\mathbf{P}}$ & $\mathbf{T}_{\mathtt{R}}$

In [ ]:
Tr_sol, P_sol, = functions.transform_matrices_in_dfs(H,T,R, Tr, P)

In [ ]:
Tr_sol #print results

In [ ]:
P_sol

#### First constrain Eq. 17b)


##### left-hand side

In [ ]:
np.sum(Hr.values,axis=1)

In [ ]:
np.sum(Hr.values,axis=1) * P_sol.values

In [ ]:
np.sum(Hr.values,axis=1) * P_sol.values * np.transpose(Ht.values)

##### rigth-hand side

In [ ]:
(Tr_sol.values @ np.transpose(Hr.values))

In [ ]:
(Tr_sol.values @ np.transpose(Hr.values)) * np.transpose(Ht.values) * P_sol.values

#### Second constraint (Eq. 17c)

In [ ]:
P_sol.values @ Hc.values

#### Third constraint (Eq. 17d)

In [ ]:
sum(Tr_sol.values)

## Diagnostic function (step: F. of the methodology $\mathfrak{M}()$)

In [ ]:
results = functions.DiagnosticFunction(H, C, P_sol, Tr_sol)

## Plot results

In [ ]:
for terminal, elems in Tr_sol.iterrows():
    for i,e in elems.items():
        if(e==1):
            ind = elements[elements["name"]==i].index[0]
            elements.at[ind, 'terminal'] = terminal

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely import Point, LineString
from shapely.ops import nearest_points

connections = gpd.GeoDataFrame(columns=["from_id", "to_id", "terminal", "coor"])

estimated_optimal_paths = [p for p in P_sol.columns.values if P_sol[p][0] == 1]
pp = [p.get_elements_names() for p in paths]
estimated_optimal_paths = [pp[int(i[1:])-1] for i in estimated_optimal_paths]
for h in estimated_optimal_paths[:-1]:
    for i in range(len(h)-1):
        elem_from = elements[elements["name"]==h[i]]
        elem_to = elements[elements["name"]==h[i+1]]

        
        coor_from = elem_from.coor.values[0]
        coor_to = elem_to.coor.values[0]

        # Calculate nearest points (if necessary, otherwise use coordinates directly)
        nearest_point_from = nearest_points(coor_from, coor_to)[0]
        nearest_point_to = nearest_points(coor_to, coor_from)[0]

        l = LineString([nearest_point_from, nearest_point_to])

        new_row = {
                        'from_id': elem_from.name.values[0], 
                        'to_id': elem_to.name.values[0], 
                        'terminal': h[-1], 
                        'coor': l
                    }
        new_row = gpd.GeoDataFrame([new_row])
        connections = pd.concat([connections, new_row], ignore_index=True)
connections = connections.set_geometry('coor')

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define a color map based on unique values in 'terminal'
unique_terminals = elements['terminal'].unique()
colors = plt.cm.get_cmap('tab20', len(unique_terminals))  # Use a colormap with a number of colors equal to unique terminals

# Create a dictionary mapping terminal values to colors
color_map = {terminal: colors(i) for i, terminal in enumerate(unique_terminals)}

# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 10))

# Plot elements with colors based on 'terminal'
elements['color'] = elements['terminal'].map(color_map)
elements.plot(ax=ax, color=elements['color'], markersize=50, label='Elements')

# Plot connections (assuming connections is a GeoDataFrame with 'coor' geometry)
connections['color'] = connections['terminal'].map(color_map)
connections.plot(ax=ax, color=connections['color'], linestyle='--', linewidth=.5, label='Connections')

# Add legend for elements
# Manually create a legend for terminal values
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor=color_map[terminal], markersize=10, label=f'Terminal {terminal}') 
                   for terminal in unique_terminals]
# ax.legend(handles=legend_elements, title='Terminals')

# Add labels and title
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Elements and Connections Plot')

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
connections